In [93]:
# !pip install category_encoders


     -------------------------------------- 81.9/81.9 kB 416.4 kB/s eta 0:00:00


In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)

import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder
from sklearn.model_selection import train_test_split
from category_encoders import TargetEncoder


# Modelos
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.inspection import permutation_importance

from sklearn import svm
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression

import xgboost as xgb

from sklearn.neural_network import MLPClassifier

In [3]:
dados = pd.read_csv("../Desafio_interno_visagio/Data/Dataset_Treino.csv")
resp = pd.read_csv("../Desafio_interno_visagio/Data/Dataset_Resposta.csv")
sample = pd.read_csv("../Desafio_interno_visagio/Data/Sample_Submission.csv")

## Limpeza dos dados

- Verificando dados ausentes

In [10]:
dados.isna().sum().sort_values(ascending=False)

Municipio                  5
ID_Aluno                   0
Disponibilidade_Tutoria    0
Dias_Espera_Inicio         0
Dias_Espera_Aprovacao      0
Data_Inscrição             0
Horario_Estudando          0
Conheceu_PROA              0
Renda_Familiar             0
Pessoas_Casa               0
Disponibilidade_3_Meses    0
Recursos                   0
Idade                      0
Aprender_EAD               0
Concluiu_EAD               0
Estudando                  0
Trabalhando                0
Estado                     0
Escolaridade               0
Tipo_escola                0
Abandono_curso             0
dtype: int64

In [9]:
resp.isna().sum().sort_values(ascending=False)

Municipio                  6
ID_Aluno                   0
Disponibilidade_Tutoria    0
Dias_Espera_Aprovacao      0
Data_Inscrição             0
Horario_Estudando          0
Conheceu_PROA              0
Renda_Familiar             0
Pessoas_Casa               0
Disponibilidade_3_Meses    0
Recursos                   0
Idade                      0
Aprender_EAD               0
Concluiu_EAD               0
Estudando                  0
Trabalhando                0
Estado                     0
Escolaridade               0
Tipo_escola                0
Dias_Espera_Inicio         0
dtype: int64

- Tratando dados ausentes

In [8]:

# conjunto de treinamento
atributos_com_nulos = ['Estudando', 'Recursos']
for col in atributos_com_nulos:
    valor_mais_frequente = dados[col].mode()[0]
    dados[col] = dados[col].fillna(valor_mais_frequente)

# conjunto de teste
atributos_com_nulos = ['Estudando', 'Concluiu_EAD', 'Recursos']
for col in atributos_com_nulos:
    valor_mais_frequente = resp[col].mode()[0]
    resp[col] = resp[col].fillna(valor_mais_frequente)

- Removendo as colunas **Conheceu_PROA** e **Municipio**

In [11]:
dados.drop(labels=['Conheceu_PROA', 'Municipio'], axis=1, inplace=True)
resp.drop(labels=['Conheceu_PROA', 'Municipio'], axis=1, inplace=True)

- Tratando dados datetime

In [12]:
dados['Data_Inscrição'] = pd.to_datetime(dados['Data_Inscrição'].str.strip(), format='%d/%m/%Y')
dados['Data_Inscrição'] = dados['Data_Inscrição'].apply(lambda x: int(x.timestamp() * 1000))

In [13]:
resp['Data_Inscrição'] = pd.to_datetime(resp['Data_Inscrição'].str.strip(), format='%d/%m/%Y')
resp['Data_Inscrição'] = resp['Data_Inscrição'].apply(lambda x: int(x.timestamp() * 1000))

### Escalonamento

- Tipos de dados

> ID_Aluno  
> Data_Inscrição  

Atributos binários:  
> Tipo_escola  
> Trabalhando  
> Estudando  
> Disponibilidade_Tutoria  
> Disponibilidade_3_Meses  

Atributos numéricos:  
> Idade  
> Pessoas_Casa  
> Dias_Espera_Aprovacao  
> Dias_Espera_Inicio   

Atributos categóricos:  
> Horario_Estudando (Nominal)  
> Conheceu_PROA (Nominal)    
> Municipio (Nominal)  
> Estado (Nominal)    
> Concluiu_EAD (Nominal)  
> Renda_Familiar (Ordinal)  
> Escolaridade (Ordinal)  
> Aprender_EAD (Ordinal)  
> Recursos (Ordinal)  

#### Atributos binários

- Escalonar os atributos binários

In [19]:
dados['Tipo_escola'].replace(to_replace=['public', 'scholarship'], value=[0, 1], inplace=True)
dados['Trabalhando'].replace(to_replace=['Não', 'Sim'], value=[0, 1], inplace=True)
dados['Estudando'].replace(to_replace=['Não', 'Sim'], value=[0, 1], inplace=True)
dados['Disponibilidade_Tutoria'].replace(to_replace=['Não', 'Sim'], value=[0, 1], inplace=True)
dados['Disponibilidade_3_Meses'].replace(to_replace=['Não', 'Sim'], value=[0, 1], inplace=True)


In [20]:
resp['Tipo_escola'].replace(to_replace=['public', 'scholarship'], value=[0, 1], inplace=True)
resp['Trabalhando'].replace(to_replace=['Não', 'Sim'], value=[0, 1], inplace=True)
resp['Estudando'].replace(to_replace=['Não', 'Sim'], value=[0, 1], inplace=True)
resp['Disponibilidade_Tutoria'].replace(to_replace=['Não', 'Sim'], value=[0, 1], inplace=True)
resp['Disponibilidade_3_Meses'].replace(to_replace=['Não', 'Sim'], value=[0, 1], inplace=True)


#### Atributos numéricos

- Tratando os dados do atributo **Pessoas_Casa**

In [21]:
dados['Pessoas_Casa'].replace(to_replace=['Mais que 10'], value=[11], inplace=True)
resp['Pessoas_Casa'].replace(to_replace=['Mais que 10'], value=[11], inplace=True)

In [22]:
dados['Pessoas_Casa'] = dados['Pessoas_Casa'].astype(int)

- Inicializando o objeto **transformador**

In [23]:
transformador = MinMaxScaler().fit(dados[['Idade',
'Pessoas_Casa',
'Dias_Espera_Aprovacao',
'Dias_Espera_Inicio']])

- Escalonando dados de treino

In [24]:
dados_num_escalonados = transformador.transform(dados[['Idade',
'Pessoas_Casa',
'Dias_Espera_Aprovacao',
'Dias_Espera_Inicio']])

X_treino_num_proc = pd.DataFrame(dados_num_escalonados, 
                                      columns=['Idade',
                                                'Pessoas_Casa',
                                                'Dias_Espera_Aprovacao',
                                                'Dias_Espera_Inicio'],
                                        index=dados.index)

- Escalonando dados de teste

In [25]:
dados_num_escalonados = transformador.transform(resp[['Idade',
'Pessoas_Casa',
'Dias_Espera_Aprovacao',
'Dias_Espera_Inicio']])

X_validador_num_proc = pd.DataFrame(dados_num_escalonados, 
                                      columns=['Idade',
                                        'Pessoas_Casa',
                                        'Dias_Espera_Aprovacao',
                                        'Dias_Espera_Inicio'],
                                        index=resp.index)

#### Atributos categóricos

- Cardinalidade dos atributos categóricos

In [18]:
dados[[
     'Horario_Estudando',
 'Estado',
 'Concluiu_EAD',
 'Renda_Familiar',
 'Escolaridade',
 'Aprender_EAD',
 'Recursos'
]].nunique().sort_values(ascending=False)

Recursos             51
Renda_Familiar        6
Horario_Estudando     5
Concluiu_EAD          5
Escolaridade          5
Estado                4
Aprender_EAD          4
dtype: int64

- Ordinal na **Renda_Familiar**

In [26]:
# Mapeamento personalizado
mapeamento_personalizado = {
    'Até 1 salário mínimo (até R$1.100)':0.1,
 'Entre 1 e 2 salários mínimos (R$1.100 – R$2.200)':0.2,
 'Entre 2 e 3 salários mínimos (R$2.200 – R$3.300)':0.3,
 'Entre 4 e 5 salários mínimos (R$4.400 – R$5.500)':0.5,
 'Entre 3 e 4 salários mínimos (R$3.300 – R$4.400)':0.4,
 'Mais que 5 salários mínimos (mais que R$5.500)':0.6
}

# Aplicar a codificação usando o mapeamento
dados['Renda_Familiar'] = dados['Renda_Familiar'].map(mapeamento_personalizado)
resp['Renda_Familiar'] = resp['Renda_Familiar'].map(mapeamento_personalizado)

- Ordinal na **Escolaridade**

In [27]:
# Mapeamento personalizado
mapeamento_personalizado = {
'Cursando o 3º ano do Ensino Médio' :0.1,
'Cursando o Ensino Superior':0.4,
 'Ensino Médio concluído e não estudando' :0.2,
 'Ensino Médio concluído':0.3,
 'Ensino Superior concluído':0.5
}

# Aplicar a codificação usando o mapeamento
dados['Escolaridade'] = dados['Escolaridade'].map(mapeamento_personalizado)
resp['Escolaridade'] = resp['Escolaridade'].map(mapeamento_personalizado)

- Ordinal na **Aprender_EAD**

In [28]:
# Mapeamento personalizado
mapeamento_personalizado = {
'Não sei dizer':0.3,
 'Muito eu tenho uma rotina definida para participar de cursos a distância':0.4,
 'Eu prefiro cursos presenciais' :0.1,
 'Quase nada':0.2
}

# Aplicar a codificação usando o mapeamento
dados['Aprender_EAD'] = dados['Aprender_EAD'].map(mapeamento_personalizado)
resp['Aprender_EAD'] = resp['Aprender_EAD'].map(mapeamento_personalizado)

- Aplicando OrdinalEncoder 

In [29]:
# Inicializar o TargetEncoder
encoder = OrdinalEncoder()

colunas_a_codificar = [['Horario_Estudando',
'Estado',
'Concluiu_EAD',
'Recursos']]

# Iterar sobre as colunas especificadas e codificar cada uma
for coluna in colunas_a_codificar:
    dados[coluna] = encoder.fit_transform(dados[coluna])


for coluna in colunas_a_codificar:
    resp[coluna] = encoder.fit_transform(resp[coluna])

- Fazendo o merge dos atributos sem os atributos **ID_Aluno** e **Data_inscrição**

In [30]:
X_treino_escalonado = pd.merge(
    dados[[
 'Tipo_escola',
 'Trabalhando',
 'Estudando',
 'Disponibilidade_Tutoria',
 'Disponibilidade_3_Meses' 
    ]].reset_index(),
    X_treino_num_proc.reset_index()
)


X_treino_escalonado1 = pd.merge(
    X_treino_escalonado,
    dados['Renda_Familiar'].reset_index()
)


X_treino_escalonado2 = pd.merge(
    X_treino_escalonado1,
    dados['Escolaridade'].reset_index()
)


X_treino_escalonado3 = pd.merge(
    X_treino_escalonado2,
    dados['Aprender_EAD'].reset_index()
)


X_treino_escalonado5 = pd.merge(
    X_treino_escalonado3,
    dados[coluna].reset_index()
)



X_treino_escalonado8 = pd.merge(
    X_treino_escalonado5,
    dados['Data_Inscrição'].reset_index()
)

X_treino_escalonado8.drop(labels=['index'], inplace=True, axis=1)

In [31]:
X_validador_escalonado = pd.merge(
    resp[[
 'Tipo_escola',
 'Trabalhando',
 'Estudando',
 'Disponibilidade_Tutoria',
 'Disponibilidade_3_Meses' 
    ]].reset_index(),
    X_validador_num_proc.reset_index()
)


X_validador_escalonado1 = pd.merge(
    X_validador_escalonado,
    resp['Renda_Familiar'].reset_index()
)


X_validador_escalonado2 = pd.merge(
    X_validador_escalonado1,
    resp['Escolaridade'].reset_index()
)


X_validador_escalonado3 = pd.merge(
    X_validador_escalonado2,
    resp['Aprender_EAD'].reset_index()
)


X_validador_escalonado5 = pd.merge(
    X_validador_escalonado3,
    resp[coluna].reset_index()
)



X_validador_escalonado8 = pd.merge(
    X_validador_escalonado5,
    resp['Data_Inscrição'].reset_index()
)


X_validador_escalonado8.drop(labels=['index'], axis=1, inplace=True)

#### Dados ausentes

In [43]:
# Calcule a média das colunas
media_das_colunas = X_treino_escalonado8['Estudando'].median()

# Substitua os valores ausentes das colunas pela média respectiva
X_treino_escalonado8['Estudando'] = X_treino_escalonado8['Estudando'].fillna(media_das_colunas)

## Modelo

In [32]:
from skopt import BayesSearchCV
from skopt.space import Real, Integer

In [ ]:
615
# 123 iterações = 10 minutos e 28.6 segundos
# 4375000 iterações totais
# 10 iterações = 14.7 segundos
search_space = {
    'max_depth': Integer(8,20),
    'learning_rate': Real(0.001, 1.0, prior='log-uniform'),
    'subsample': Real(0.5, 1.0),
    'colsample_bytree': Real(0.5, 1.0),
    'colsample_bylevel': Real(0.5, 1.0),
    'colsample_bynode' : Real(0.5, 1.0),
    'reg_alpha': Real(0.0, 10.0),
    'reg_lambda': Real(0.0, 10.0),
    'gamma': Real(0.0, 10.0)
}

ftwo_scorer = make_scorer(fbeta_score, beta=0.5)

opt = BayesSearchCV(xgb.XGBClassifier(), search_space, cv=3, n_iter=123, scoring= ftwo_scorer, 
                    random_state=8).fit(X_treino_escalonado8, dados['Abandono_curso'])


In [50]:
opt.best_params_

OrderedDict([('colsample_bylevel', 1.0),
             ('colsample_bynode', 0.6555590530528685),
             ('colsample_bytree', 1.0),
             ('gamma', 2.5312720195633127),
             ('learning_rate', 1.0),
             ('max_depth', 20),
             ('reg_alpha', 10.0),
             ('reg_lambda', 4.332080712683039),
             ('subsample', 0.654538364129673)])

In [51]:
opt.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1.0,
              colsample_bynode=0.6555590530528685, colsample_bytree=1.0,
              enable_categorical=False, gamma=2.5312720195633127, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=1.0, max_delta_step=0, max_depth=20,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, predictor='auto',
              random_state=0, reg_alpha=10.0, reg_lambda=4.332080712683039,
              scale_pos_weight=1, subsample=0.654538364129673,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [58]:
modelo = xgb.XGBClassifier(random_state=8, colsample_bylevel= 1.0,
             colsample_bynode= 0.6555590530528685,
             colsample_bytree= 1.0,
             gamma= 2.5312720195633127,
             learning_rate= 1.0,
             max_depth= 20,
             reg_alpha= 10.0,
             reg_lambda= 4.332080712683039,
             subsample= 0.654538364129673)


"""
colsample_bylevel= 1.0,
             colsample_bynode= 0.6555590530528685,
             colsample_bytree= 1.0,
             gamma= 2.5312720195633127,
             learning_rate= 1.0,
             max_depth= 20,
             reg_alpha= 10.0,
             reg_lambda= 4.332080712683039,
             subsample= 0.654538364129673



base_score=0.5, booster='gbtree', colsample_bylevel=1.0,
              colsample_bynode=0.6555590530528685, colsample_bytree=1.0,
              enable_categorical=False, gamma=2.5312720195633127, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=1.0, max_delta_step=0, max_depth=20,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, predictor='auto',
              random_state=0, reg_alpha=10.0, reg_lambda=4.332080712683039,
              scale_pos_weight=1, subsample=0.654538364129673,
              tree_method='exact', validate_parameters=1, verbosity=None
"""

"\ncolsample_bylevel= 1.0,\n             colsample_bynode= 0.6555590530528685,\n             colsample_bytree= 1.0,\n             gamma= 2.5312720195633127,\n             learning_rate= 1.0,\n             max_depth= 20,\n             reg_alpha= 10.0,\n             reg_lambda= 4.332080712683039,\n             subsample= 0.654538364129673\n\n\n\nbase_score=0.5, booster='gbtree', colsample_bylevel=1.0,\n              colsample_bynode=0.6555590530528685, colsample_bytree=1.0,\n              enable_categorical=False, gamma=2.5312720195633127, gpu_id=-1,\n              importance_type=None, interaction_constraints='',\n              learning_rate=1.0, max_delta_step=0, max_depth=20,\n              min_child_weight=1, missing=nan, monotone_constraints='()',\n              n_estimators=100, n_jobs=8, num_parallel_tree=1, predictor='auto',\n              random_state=0, reg_alpha=10.0, reg_lambda=4.332080712683039,\n              scale_pos_weight=1, subsample=0.654538364129673,\n              t

In [59]:
modelo.fit(X_treino_escalonado8, dados['Abandono_curso'])

c:\Users\jeffe\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:49:53] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1.0,
              colsample_bynode=0.6555590530528685, colsample_bytree=1.0,
              enable_categorical=False, gamma=2.5312720195633127, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=1.0, max_delta_step=0, max_depth=20,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, predictor='auto',
              random_state=8, reg_alpha=10.0, reg_lambda=4.332080712683039,
              scale_pos_weight=1, subsample=0.654538364129673,
              tree_method='exact', validate_parameters=1, verbosity=None)

### Gerando o submission

In [60]:
df = pd.DataFrame(
    {'ID_Aluno': resp.ID_Aluno,
     'Abandono_curso': modelo.predict(X_validador_escalonado8) }
)

In [61]:
df.to_csv("submission_xgboost_otimizado7.csv", index=False)

In [62]:
ver = pd.read_csv("submission_xgboost_otimizado7.csv")

In [63]:
ver.Abandono_curso.value_counts()

1    1155
0      83
Name: Abandono_curso, dtype: int64